In [8]:
import numpy as np
from utils.pose_evaluation_utils import inv, pose_vec_q_to_mat
import pandas as pd

def ominus(a,b):
    """
    Compute the relative 3D transformation between a and b.

    Input:
    a -- first pose (homogeneous 4x4 matrix)
    b -- second pose (homogeneous 4x4 matrix)

    Output:
    Relative 3D transformation from a to b.
    """
    return np.dot(np.linalg.inv(a),b) 

def compute_distance(transform):
    """
    Compute the distance of the translational component of a 4x4 homogeneous matrix.
    """
    return np.linalg.norm(transform[0:3,3])

def compute_angle(transform):
    """
    Compute the rotation angle from a 4x4 homogeneous matrix.
    """
    # an invitation to 3-d vision, p 27
    return np.arccos( min(1,max(-1, (np.trace(transform[0:3,0:3]) - 1)/2) ))

def find_rpe(gt,pred):
    error44 = ominus(ominus( pred[0], pred[1] ),
                            ominus( gt[0], gt[1] ) )
    trans = compute_distance(error44)
    rot = compute_angle(error44)    

    return error44,rot,trans

def find_all_rpe(gts,preds):
    assert len(gts) == len(preds), "The data from preds and gts doesn't match"
    errors, err_t, err_r = [], [], []
    for i in range(1,len(gts)):
        t1,t2,t3 = find_rpe([gts[i-1],gts[i]],[preds[i-1],preds[i]])
        errors.append(t1)
        err_t.append(t2)
        err_r.append(t3)
    errors = np.array(errors)
    err_t = np.array(err_t)
    err_r = np.array(err_r)
    rmse_t = np.sqrt(np.dot(err_t,err_t) / len(err_t))
    rmse_r = np.sqrt(np.dot(err_r,err_r) / len(err_r))
    return rmse_r,rmse_t

def align_to_origin(data,is_vo=False):
    viewer_origin = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])
    vo_origin = np.array([[0,0,1,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,1]])

    if is_vo:
        data = vo_origin@data
    else:
        data = viewer_origin@inv(data[0])@data

    return data

def align(model,data):
    """Align two trajectories using the method of Horn (closed-form).
    
    Input:
    model -- first trajectory (3xn)
    data -- second trajectory (3xn)
    
    Output:
    rot -- rotation matrix (3x3)
    trans -- translation vector (3x1)
    trans_error -- translational error per point (1xn)
    
    """
    model_zerocentered = model - model.mean(1)
    data_zerocentered = data - data.mean(1)
    
    W = np.zeros( (3,3) )
    for column in range(model.shape[1]):
        W += np.outer(model_zerocentered[:,column],data_zerocentered[:,column])
    U,d,Vh = np.linalg.linalg.svd(W.transpose())
    S = np.matrix(np.identity( 3 ))
    if(np.linalg.det(U) * np.linalg.det(Vh)<0):
        S[2,2] = -1
    rot = U*S*Vh
    trans = data.mean(1) - rot * model.mean(1)
    
    model_aligned = rot * model + trans
    alignment_error = model_aligned - data
    
    trans_error = np.sqrt(np.sum(np.multiply(alignment_error,alignment_error),0)).A[0]

    return model_aligned,trans_error


def txt_to_4x4(fname,datas=-1):
    data = np.array(pd.read_csv(fname,delimiter=' '))[:datas]
    data = np.array([pose_vec_q_to_mat(i) for i in data])
    return data

def find_ate(gts,preds):

    gt, pred = gts[:,:3,-1],preds[:,:3,-1]

    offset = gt[0] - pred[0]

    pred += offset[None,:]

    # Optimize the scaling factor
    scale = np.sum(gt * pred)/np.sum(pred ** 2)
    pred = pred*scale
    alignment_error = pred - gt

    #average translation error
    rmse = np.sqrt(np.sum(alignment_error ** 2))/len(gt)

    preds[:,:3,-1] = pred
    return preds,rmse

def compute_error(gt_path,pred_path):
    gts = txt_to_4x4(gt_path)
    gts = align_to_origin(gts)
    preds = txt_to_4x4(pred_path)
    preds = align_to_origin(preds,True)

    preds_scaled,ate = find_ate(gts,preds)
    rpe = find_all_rpe(gts,preds_scaled)

    print(ate,rpe)
    return preds, preds_scaled

def compute_error_TUM(gt_path,pred_path):
    gts = txt_to_4x4(gt_path)
    gts = align_to_origin(gts)
    preds = txt_to_4x4(pred_path)
    
    p, trans_err = align(np.matrix(preds[:,:3,3].T),np.matrix(gts[:,:3,3].T))
    pred_scaled = np.copy(preds)
    pred_scaled[:,:3,3] = p.T
    print("TransErr = ", np.sqrt(np.dot(trans_err,trans_err) / len(trans_err)))

    _,ate = find_ate(gts,pred_scaled)
    rpe = find_all_rpe(gts,pred_scaled)

    print(ate,rpe)
    return preds, pred_scaled

''' 
Display various stats using Translational and Rotational error

print "compared_pose_pairs %d pairs"%(len(trans_error))

print "translational_error.rmse %f m"%numpy.sqrt(numpy.dot(trans_error,trans_error) / len(trans_error))
print "translational_error.mean %f m"%numpy.mean(trans_error)
print "translational_error.median %f m"%numpy.median(trans_error)
print "translational_error.std %f m"%numpy.std(trans_error)
print "translational_error.min %f m"%numpy.min(trans_error)
print "translational_error.max %f m"%numpy.max(trans_error)

print "rotational_error.rmse %f deg"%(numpy.sqrt(numpy.dot(rot_error,rot_error) / len(rot_error)) * 180.0 / numpy.pi)
print "rotational_error.mean %f deg"%(numpy.mean(rot_error) * 180.0 / numpy.pi)
print "rotational_error.median %f deg"%(numpy.median(rot_error) * 180.0 / numpy.pi)
print "rotational_error.std %f deg"%(numpy.std(rot_error) * 180.0 / numpy.pi)
print "rotational_error.min %f deg"%(numpy.min(rot_error) * 180.0 / numpy.pi)
print "rotational_error.max %f deg"%(numpy.max(rot_error) * 180.0 / numpy.pi)
'''
1

1

In [9]:
gt_path = './data/Trajs/abandonedfactory_night/P001.txt'
pred_path = './data/traj_Factory_night_0001.txt'

bp, preds = compute_error_TUM(gt_path, pred_path)   # Less Loss, Less Sense
bp2,preds2= compute_error(gt_path, pred_path)       # More Loss, better sence   # Using this

TransErr =  39.20777658325479
0.3231317099357854 (0.3141236196159193, 0.03918972871547101)
0.5103990822988096 (0.3778498094956221, 0.03918972871547116)


In [10]:
import numpy as np

from utils.pose_helper import make_intrinsic, Traj_helper
from utils.viz_helper import *

helper = Traj_helper()
window = viewer()
window.run()
display_poses_dataset(window,f_name='./data/Trajs/abandonedfactory_night/P001.txt',color=[0,0.5,0],data_origin=True)
display_poses(window,preds,color=[1,0,0])
# display_poses(window,bp,color=[0.5,0,0])
display_poses(window,preds2,color=[0,0,1])
# display_poses(window,bp2,color=[0,0,0.5])

window.vis.run()